# Agent Workflow

In [ ]:
# imports
import os
import google.generativeai as genai
import geopandas as gpd
import shapely
from pathlib import Path
from dotenv import load_dotenv
from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, GeometryCollection

from utils import cut_polygon
from agent_toolkit import (
    AgentOrchestrator,
    ReasoningAgent,
    ValidationAgent,
    faiss_search,
    faiss_search_flatten,
    gpkg_query,
)
from agent_prompts import SYSPROMPT_REASONING_AGENT, SYSPROMPT_VALIDATION_AGENT

load_dotenv()

### loading RAG and GEO databases

In [ ]:
# load FAISS database
EMB_DIM = 3072
EMBEDDER = "text-embedding-3-large"

FAISS_INDEX_DIR = "data/spatial_genai_storage/database_RAG"
FAISS_META_PATH = Path(FAISS_INDEX_DIR) / "metadata.json"
FAISS_INDEX_PATH = str(Path(FAISS_INDEX_DIR) / "faiss.index")

In [ ]:
# load geopackage data

GPKG_FILE_PATH = 'data/spatial_genai_storage/data_PDOK/top10nl_Compleet.gpkg'
BBOX_UTRECHT_PROV = (109311, 430032, 169326, 479261)

In [ ]:
# grab the province of Utrecht polygon as base polygon
base_polygon = gpkg_query(
    table_name='top10nl_registratief_gebied_vlak',
    bbox=BBOX_UTRECHT_PROV,
    filters={'typeregistratiefgebied': 'provincie', 'naamnl': 'Utrecht'}  # Adjust column if name is in 'naamofficieel'
)
base_geom = base_polygon.geometry.iloc[0]  # Extract Shapely geometry

### Setting up API calling

In [ ]:
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

### Basic workflow

In [ ]:
# Get column:feature filters plus reasoning
NUM_ROUNDS = 2

# basic inputs
thematic_object = "windturbine"
object_description = "Er is uitgegaan van een 'referentie windturbinetype'. Uitgangspunt is dat het referentie windturbinetype een ashoogte van 160 meter, rotordiameter van 162 meter en tiphoogte van 241 meter heeft."
base_polygon_name = "Provincie Utrecht"

# get judicial references from RAG database
query = f"Vind juridische belemmeringen voor het plaatsen van een {thematic_object} met de volgende kenmerken: {object_description}. Geef relevante wet- en regelgeving, beleidsdocumenten en ruimtelijke plannen binnen {base_polygon_name}."
judicial_reference = faiss_search(query=query, k=30, location="Utrecht")
judicial_reference = faiss_search_flatten(judicial_reference)

# get database reference from pre-made file
database_reference_path = "data/spatial_genai_storage/data_PDOK/llm_reference.txt"
with open(database_reference_path, "r", encoding="utf-8") as f:
    database_reference = f.read()

# prepare agent ecosystem
reasoning_agent = ReasoningAgent(
    name="inhoudsexpert",
    system_prompt_template=SYSPROMPT_REASONING_AGENT,
)
validation_agent = ValidationAgent(
    name="kwaliteitscontroleur",
    system_prompt_template=SYSPROMPT_VALIDATION_AGENT,
)
orchestrator = AgentOrchestrator(agents=[reasoning_agent, validation_agent])

# context replacement happens through agent_toolkit.prepare_system_prompt()
agent_context = {
    "thematic_object": thematic_object,
    "object_description": object_description,
    "judicial_reference": judicial_reference,
    "database_reference": database_reference,
    "base_polygon_name": base_polygon_name,
    "feedback": [],
}

agent_rounds = orchestrator.run_rounds(agent_context, num_rounds=NUM_ROUNDS)
latest_round = agent_rounds[-1]
filters_plus_reasoning = latest_round["reasoning"]["raw"]
filters = latest_round["reasoning"]["parsed"]
validation_summary = latest_round["validation"]["parsed"]

In [ ]:
# Inspect validation feedback from the latest round
validation_summary

In [ ]:
print(filters_plus_reasoning)

### Start processing (takes a while, 15-20min... with bigger boundary box 35-45min)

In [ ]:
# Cut from province polygon using json format filters

categories = {
    "harde belemmering": [],
    "complexe belemmering": [],
    "zachte belemmering": []
}

for item in filters:
    categorie = item.get("categorie", "").lower()
    if categorie in categories:
        categories[categorie].append(item)
        
# Store cut geometries per category
cut_areas_by_category = {cat: [] for cat in categories.keys()}

# Process all filters and apply to base geometry
for item in filters:
    tabel = item["tabel"]
    kolom = item["kolom"]
    waarde = item["waarde"]
    categorie = item.get("categorie", "").lower()

    print(f"Processing filter: {tabel}.{kolom} = '{waarde}'")
    cut_gdf = gpkg_query(table_name=tabel, filters={kolom: waarde})
    
    if not cut_gdf.empty:
        cut_geoms = cut_gdf.geometry.tolist()
        union_cut = unary_union(cut_geoms)
        
        if categorie in cut_areas_by_category:
            cut_areas_by_category[categorie].append(union_cut)
        
        base_geom = cut_polygon(base_geom, union_cut)
        
        print(f"Cut applied for {tabel}.{kolom}='{waarde}' - {len(cut_gdf)} polygons removed.")
    else:
        print(f"No polygons found for {tabel}.{kolom}='{waarde}' - skipping.")

# Create new GeoDataFrame with cut geometry
cut_gdf_result = gpd.GeoDataFrame(
    {"name": ["Utrecht_Province_Cut"], "description": ["Remaining area after all cuts"]},
    geometry=[base_geom],
    crs=base_polygon.crs
)

# Optional: Save to file
output_file = "utrecht_cut_with_categories.gpkg"
cut_gdf_result.to_file(output_file, layer="remaining_area", driver="GPKG")
print(f"Remaining area saved to {output_file}")

In [ ]:
output_file = "utrecht_cut_with_categories.gpkg"
cut_gdf_result.to_file(output_file, layer="remaining_area", driver="GPKG")
print(f"Remaining area saved to {output_file}")

In [ ]:
# Save cut areas by category as separate layers
for categorie, cut_geoms in cut_areas_by_category.items():
    if not cut_geoms:
        print(f"No cut areas for category: {categorie}")
        continue
    
    # Union all cut geometries for this category
    category_union = unary_union(cut_geoms)
    
    if category_union.is_empty:
        print(f"Empty geometry for category: {categorie}")
        continue
    
    # Create layer name
    layer_name = f"cut_{categorie.replace(' ', '_')}"
    
    # Create GeoDataFrame for cut areas
    cut_area_gdf = gpd.GeoDataFrame(
        {
            "name": [f"Cut areas - {categorie}"],
            "description": [f"Areas removed due to {categorie}"],
            "categorie": [categorie]
        },
        geometry=[category_union],
        crs=base_polygon.crs
    )
      
    # Save to GeoPackage
    cut_area_gdf.to_file(output_file, layer=layer_name, driver="GPKG", mode='a')
    print(f"Saved cut areas for '{categorie}' as layer '{layer_name}'")

print(f"\nAll layers saved to {output_file}")

In [ ]:
# EXPERIMENTAL, SIMPLIFY INCLUDING LAYERS

# Simplify and validate all layers (remaining area + cut areas by category)

input_file = "utrecht_cut_with_categories.gpkg"
output_file = "utrecht_cut_with_categories_simplified_2.gpkg"

# Get all layers from the input GeoPackage
layers_df = gpd.list_layers(input_file)
print(f"Found {len(layers_df)} layers")
print(f"Layer info:\n{layers_df}")

# Extract layer names from the 'name' column
layer_names = layers_df['name'].tolist()
print(f"\nLayer names: {layer_names}")

for idx, layer_name in enumerate(layer_names):
    print(f"\n=== Processing layer {idx+1}/{len(layer_names)}: {layer_name} ===")
    
    try:
        # 1) Read the layer
        gdf = gpd.read_file(input_file, layer=layer_name)
        
        if gdf.empty:
            print(f"Layer {layer_name} is empty, skipping")
            continue
        
        assert gdf.crs is not None, f"CRS missing for layer {layer_name}"
        
        # 2) Ensure polygonal, valid parts only
        gdf["geometry"] = gdf.geometry.apply(shapely.make_valid)
        gdf = gdf[~gdf.geometry.is_empty & gdf.geometry.notna()]
        
        if gdf.empty:
            print(f"No valid geometries in layer {layer_name}, skipping")
            continue
        
        # 3) Explode geometry collections
        gdf = gdf.explode(ignore_index=True)
        
        # 4) Keep only polygonal types
        gdf = gdf[gdf.geom_type.isin(["Polygon", "MultiPolygon"])]
        
        if gdf.empty:
            print(f"No polygonal geometries remain for layer {layer_name}")
            continue
        
        # 5) Union into a single polygonal geometry
        poly_union = unary_union(gdf.geometry.values)
        
        def polygons_only(geom):
            if geom.is_empty:
                return geom
            if isinstance(geom, (Polygon, MultiPolygon)):
                return geom
            if isinstance(geom, GeometryCollection):
                polys = [g for g in geom.geoms if isinstance(g, (Polygon, MultiPolygon))]
                if not polys:
                    return shapely.geometry.GeometryCollection()
                return unary_union(polys)
            return shapely.geometry.GeometryCollection()
        
        poly_union = polygons_only(poly_union)
        poly_union = shapely.make_valid(poly_union)
        
        if poly_union.is_empty:
            print(f"Empty geometry after processing for layer {layer_name}, skipping")
            continue
        
        if isinstance(poly_union, Polygon):
            poly_union = MultiPolygon([poly_union])
        
        # 6) Remove tiny slivers (< 1 m²)
        tmp = gpd.GeoDataFrame(geometry=[poly_union], crs=gdf.crs).explode(index_parts=False)
        tmp = tmp[tmp.area > 1.0]  # keep > 1 m²
        
        if tmp.empty:
            print(f"All polygons too small for layer {layer_name}, skipping")
            continue
        
        poly_union = unary_union(tmp.geometry.values)
        
        if isinstance(poly_union, Polygon):
            poly_union = MultiPolygon([poly_union])
        
        # 7) Save the simplified layer
        # Get original attributes from first row
        attrs = gdf.iloc[0].drop('geometry').to_dict()
        
        out = gpd.GeoDataFrame(
            [attrs],
            geometry=[poly_union],
            crs=gdf.crs
        )
        
        print(f"  Geometry type: {out.geom_type.iloc[0]}")
        print(f"  Area: {out.geometry.iloc[0].area:.2f} sq meters")
        print(f"  Number of parts: {len(list(out.geometry.iloc[0].geoms)) if hasattr(out.geometry.iloc[0], 'geoms') else 1}")
        
        # Save (first layer creates file, others append)
        if idx == 0:
            out.to_file(output_file, layer=layer_name, driver="GPKG")
        else:
            out.to_file(output_file, layer=layer_name, driver="GPKG", mode='a')
        
        print(f"  ✓ Saved simplified layer '{layer_name}'")
        
    except Exception as e:
        print(f"  ✗ Error processing layer '{layer_name}': {e}")
        import traceback
        traceback.print_exc()
        continue

print(f"\n{'='*60}")
print(f"All simplified layers saved to {output_file}")
print(f"{'='*60}")

# 8) Verify all layers
print("\nVerification summary:")
print("-" * 60)
try:
    layers_df_out = gpd.list_layers(output_file)
    for layer_name in layers_df_out['name'].tolist():
        chk = gpd.read_file(output_file, layer=layer_name)
        print(f"\nLayer: {layer_name}")
        print(f"  Records: {len(chk)}")
        print(f"  Geometry type: {chk.geom_type.iloc[0]}")
        print(f"  CRS: {chk.crs}")
        print(f"  Total area: {chk.geometry.iloc[0].area:.2f} sq meters")
except Exception as e:
    print(f"Error during verification: {e}")